# Perfis de apps lucrativos para a App Store e Google Play Markets

Nosso objetivo neste projeto é encontrar perfis de aplicativos mobile que sejam lucrativos para a App Store e o Google Play. Devemos permitir que nossa equipe de desenvolvedores tome decisões baseadas em dados com relação ao tipo de aplicativo que eles criam.
Criamos apenas aplicativos gratuitos para download e instalação, e nossa principal fonte de receita para qualquer aplicativo é influenciada principalmente pelo seu número de usuários.

Nosso foco é analisar dados para ajudar os desenvolvedores a entender quais tipos de aplicativos provavelmente atrairão mais usuários.

Analisaremos dois conjuntos de dados:
   

 - Dados sobre aproximadamente dez mil aplicativos Android da Google Play. Você pode baixar o arquivo através [deste link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
 - Dados sobre aproximadamente sete mil aplicativos iOS da Apple Store. Você pode baixar o arquivo através [deste link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

## Abrindo o arquivo 

In [1]:
from csv import reader

abrir_arquivo = open('googleplaystore.csv')
ler_arquivo = reader(abrir_arquivo)
android = list(ler_arquivo)

# especificar o cabeçalho #
android_cabe = android[0]
android = android[1:]

abrir_arquivo = open('AppleStore.csv')
ler_arquivo = reader(abrir_arquivo)
ios = list(ler_arquivo)

ios_cabe = ios[0]
ios = ios[1:]


Vamos agora criar e utilizar a função `explore_data()` para facilitar nossa análise. Também acrescentaremos à essa função a opção de mostrar o número de fileiras e colunas para cada conjunto de dados (Android/iOS) 

In [2]:
def explore_data(dataset, start, end, linhas_e_colunas = False):
    parte_dataset = dataset[start:end]
    for row in parte_dataset:
        print(row)
        print('\n') 
        
    if linhas_e_colunas:
        print('Número de fileiras', len(dataset))
        print('Número de colunas', len(dataset[0]))
        
print(android_cabe)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Número de fileiras 10841
Número de colunas 13


Como é possível ver no output, algumas colunas vão nos ajudar em nossa análise, tais como `'App'`, `'Category'`, `'Rating'`, `'Installs'`,`'Price'`, `'Genres'`.
Vamos agora analisar os dados da App Store.

In [3]:
abrir_arquivo = open('AppleStore.csv')
ler_arquivo = reader(abrir_arquivo)
ios = list(ler_arquivo)

ios_cabe = ios[0]
ios = ios[1:]

print(ios_cabe)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Número de fileiras 7197
Número de colunas 16


Aqui, nos atentaremos às colunas `'track_name'`,`'currency'`, `'price'`, `'rating_count_tot'`, `'rating_count_ver'`,` 'prime_genre'`.

## Deletando dados imprecisos

Os dados da Google Play tem uma [seção de discussão](https://www.kaggle.com/lava18/google-play-store-apps/discussion) e podemos ver que em uma certa fileira, temos um erro:

In [4]:
print(android[10472]) # erro #
print('\n')
print(android_cabe) # cabeçalho #
print('\n')
print(android[0]) 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Podemos ver que a linha 10472 corresponde ao app **Life made WI-Fi Touchscreen Photo Frame** e podemos ver que a avaliação é 19. Isso é por que o limite máximo de avaliação para a Google Play é cinco estrelas , assim, vamos então deletar essa fileira.

In [5]:
print(len(android))
del android[10472] # rodar esse código uma única vez !!! #
print(len(android))


10841
10840


## Removendo entradas duplicadas

Note que, se nós explorarmos os dados da Google Play, vamos ver que alguns apps tem mais de uma 'entrada'. O Instagram, por exemplo, tem quatro entradas:

In [6]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Isso acontece também com muitos outros aplicativos.

In [7]:
apps_duplicados = []
apps_n_duplicados = []

for app in android:
    nome = app[0]
    if nome in apps_n_duplicados:
        apps_duplicados.append(nome)
    else:
        apps_n_duplicados.append(nome)

print('Número de apps duplicados:', len(apps_duplicados))
print('\n')
print('Exemplos de apps duplicados:', apps_duplicados[:15])

Número de apps duplicados: 1181


Exemplos de apps duplicados: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Nós não queremos contar um app mais de uma vez quando estivermos analizando os dados, então vamos remover as entradas duplicadas e continuar somente com uma entrada por app.

Note que a grande diferença está na quarta posição de cada fileira (Reviews). Os números mostrados foram coletados em diferentes momentos. Podemos usar isso para criar um critério! Vamos pegar as fileiras que tem maior número de reviews (porque quanto maior o número de reviews, mais confiáveis são as classificações.

Vamos então:

- Criar um dicionário onde cada chave corresponde a um app e o seu valor é o maior número de reviews desse app

- Usar o dicionário para criar outro dataset que terá somente uma entrada por app.

In [8]:
max_reviews = {}

for app in android:
    nome = app[0]
    n_reviews = float(app[3])
    
    if nome in max_reviews and max_reviews[nome] < n_reviews:
        max_reviews[nome] = n_reviews
    
    elif nome not in max_reviews:
        max_reviews[nome] = n_reviews

Anteriormente, vimos que existem 1,181 casos onde um app aparece mais de uma vez, então o tamanho do dicionário deve ser igual a diferença entre o tamanho do dataset e 1,181.

In [9]:
print('Tamanho esperado:', len(android) - 1181)
print('Tamanho atual:', len(max_reviews))

Tamanho esperado: 9659
Tamanho atual: 9659


Ok. Agora usaremos o dicionário para remover os duplicados. Para esses casos nós somente usaremos entradas com alto número de reviews. Iremos:

- Inicializar duas listas vazias: android_0 e ja_adc
- Vamos usar o looping no `android`e para cada iteração:
    - Isolamos o nome do app e o número de reviews
    - Adicionamos a fileira (`app`) à lista `android_0` e o `nome` à lista `ja_adc` se:
        - o número de reviews do app bater com o número de reviews desse app em `max_reviews`, e
        - o nome do app já não esteja na lista `ja_adc`. Precisamos acrescentar essa condição para esses casos onde o maior número de reviews de um app duplicado seja o mesmo para mais de uma entrada (exemplo: App1 tem duas entradas, e o número de reviews é o mesmo). Se somente fizermos `max_review[name] == n_reviews` continuaremos tendo entradas duplicadas para alguns apps.

In [10]:
android_0 = []
ja_adc = []

for app in android:
    nome = app[0]
    n_reviews = float(app[3])
    
    if(max_reviews[nome] == n_reviews) and (nome not in ja_adc):
        android_0.append(app)
        ja_adc.append(nome) # precisa estar dentro do if #

Vamos checar o número de fileiras então:

In [11]:
explore_data(android_0, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Número de fileiras 9659
Número de colunas 13


   ## Removendo Apps (que usam caracteres diferentes do inglês)

Entre todos esses dados, percebemos que os nomes de alguns apps sugerem que eles não são direcionados para o público que fala inglês (ou que usa os mesmos caracteres que o alfabeto inglês). Um exemplo deles são:


In [12]:
print(ios[813][1])
print(ios[6731][1])

print(android_0[4412][0])
print(android_0[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


Não temos interesse nesses aplicativos, então iremos removê-los. Uma forma de fazer isso, é remover cada app que tenha símbolos (que não estão no alfabeto inglês, ou números de 0 a 9, ou pontuações de marcação - !, . , ?, etc -  e outros símbolos permitidos - +, * , / ,  etc ) no nome.

Todos esses caracteres que estão presentes nos textos em inglês são codificados usando a tabela ASCII. Cada caractere ASCII tem um número correspondente, entre 0 e 127, associado a ele. Podemos usar isso para construir uma função que cheque o nome do aplicativo e nos diga se ele contém ou não caracteres que não estão na tabela ASCII.

In [13]:
def e_ingles(string):
    
    for caractere in string:
        if ord(caractere) > 127:
            return False
    
    return True

print(e_ingles('Instagram'))
print(e_ingles('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


Utilizamos a função `ord()` para checar o número correspondente de cada caractere.

Mas e se algum nome utilizar outros símbolos como emojis ou ™ por exemplo?

In [14]:
print(e_ingles('Docs To Go™ Free Office Suite'))
print(e_ingles('Instachat 😜'))

False
False


Para minimizar a perda de dados, somente removeremos apps se seus nomes tiverem mais que 3 caracteres fora da tabela ASCII:

In [15]:
def e_ingles(string):
    n_ascii = 0
    
    for caractere in string:
        if ord(caractere) > 127:
            n_ascii += 1
            
    if n_ascii > 3:
        return False
    else:
        return True
    
print(e_ingles('Docs To Go™ Free Office Suite'))
print(e_ingles('Instachat 😜'))

True
True


 Vamos agora utilizar a função `e_ingles` para filtrar ambos os arquivos:

In [16]:
android_ascii = []
ios_ascii = []

for app in android_0:
    nome = app[0]
    if e_ingles(nome):
        android_ascii.append(app)
        
for app in ios:
    nome = app[1]
    if e_ingles(nome):
        ios_ascii.append(app)
        
explore_data(android_ascii, 0, 3, True)
print('\n')
explore_data(ios_ascii, 0, 3, True)
    

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Número de fileiras 9614
Número de colunas 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+'

Vemos então que sobraram 9614 app Android e 6183 app iOS!

## Isolando os Apps gratuitos

A seguir vamos isolar os apps gratuitos dos dois conjuntos de dados (Android e iOS):

In [17]:
android_valor = []
ios_valor = []

for app in android_ascii:
    valor = app[7]
    if valor == '0':
        android_valor.append(app)
        
for app in ios_ascii:
    valor = app[4]
    if valor == '0.0':
        ios_valor.append(app)
        
print(len(android_valor))
print(len(ios_valor))

8864
3222


Ficamos então com 8864 apps Android e 3222 apps iOS.

## Apps mais comuns - por gênero

Como mencionado no início deste projeto, nosso objetivo é determinar tipos de apps que atraem mais usuários. Para minimizar riscos, nossa estratégia será dividida em três passos:

1. Construir uma mini versão Android do app e adicioná-la à Google Play.
2. Se o app tiver um bom feedback dos usuários, nós o desenvolveremos futuramente.
3. Se o app gera rentabilidade depois de seis meses, também construiremos sua versão iOS e a colocaremos na App Store.

Como nossa meta é adicionar tanto na App Store como no Google Play, temos que encontrar perfil que são bem sucedidos em ambas as plataformas.

Vamos começar nossa análise com os gêneros mais comuns para cada plataforma. Para isso, vamos construir uma tabela de frequência para a coluna `prime_genre` da App Store, e as colunas `Genres`e `Category` da Google Play.

Vamos contruir duas funções que podemos usar para analizar as tabelas de frequência:

   - Uma função para gerar as tabelas de frequência que mostram as porcentagens
   - Outra função que podemos usar para mostrar as porcentagens em ordem decrescente.

In [18]:
def tab_freq(dataset, index):
    tabela = {}
    total = 0
    
    for row in dataset:
        total += 1
        vale = row[index]
        
        if vale in tabela:
            tabela[vale] += 1
        else:
            tabela[vale] = 1
            
    porcentagens_tabela = {}

    for key in tabela:
        porcentagem = (tabela[key]/ total) * 100
        porcentagens_tabela[key] = porcentagem

    return porcentagens_tabela

In [19]:
def mostrar_tabela(dataset, index):
    tabela = tab_freq(dataset, index)
    tabela_mostra = []
    for key in tabela:
        key_e_uma_tupla = (tabela[key], key)
        tabela_mostra.append(key_e_uma_tupla)
        
    tabela_aleatoria = sorted(tabela_mostra, reverse = True)
    for entrada in tabela_aleatoria:
        print(entrada[1], ':', entrada[0])

Vamos examinar a frequencia para a coluna `prime_genre` da App Store:

In [20]:
mostrar_tabela(ios_valor, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Podemos ver que mais da metade é de Jogos. Mas o fato de ser o mais numeroso, não necessariamente quer dizer que eles têm o maior número de usuários. Vamos examinar a Google Play:

In [21]:
mostrar_tabela(android_valor, 1) # Categoria

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

O cenário parece totalmente diferente na Google Play. Parece que um bom número de apps foram desenvolvidos para propósitos práticos (famílias, ferramentas, negócios, estilo de vida, produtividade, etc). Se investigássemos mais a fundo, veríamos que a categoria família é composta muito mais por jogos para crianças.

Apps práticos parecem ter uma melhor representatividade na Google Play comparada à App Store.

In [22]:
mostrar_tabela(android_valor, -4) 

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Agora, queremos ter uma ideia de que tipos de apps tem a maior quantidade de usuários.

## Apps mais populares (por gênero) na App Store

   Um jeito de descobrir quais gêneros são mais populares (possuem mais usuários) é calculando a média de downloads por gênero. Para os dados da Google Play, podemos achar essa informação na coluna `Installs`, mas para os dados da App Store essa informação está faltando.
 Vamos então calcular a média de avaliações por app na Apple Store:

In [24]:
generos_ios = tab_freq(ios_valor, -5)

for genero in generos_ios:
    total = 0
    tam_genero = 0
    for app in ios_valor:
        gen_app = app[-5]
        if gen_app == genero:
            n_ava = float(app[5])
            total += n_ava
            tam_genero += 1
    media_n_ava = total / tam_genero
    print(genero, ':', media_n_ava)
            

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Em média, aplicativos como Waze e Google Maps (Navigation) tem o maior número de reviews .

In [25]:
for app in ios_valor:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # imprime nome e avaliações

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


O mesmo padrão se encontra em apps de redes sociais, que são influenciados por gigantes como Facebook, Instagram, Pinterest, etc. Isso também acontece em apps de música, por causa de aplicativos como Pandora, Spotify, Deezer e Shazam. Esses gigantes influenciam pesadamente na média que estamos calculando.

Para que possamos ter uma visão dos gêneros que realmente são mais populares, sem ter que nos preocupar tanto com a influência desses grandes apps citados, iremos removê-los de cada gênero e então recalcular as médias.

Apps de referência têm 74,942 avaliações, mas elas estão relacionadas basicamente à Bíblia e ao Dicionario :

In [26]:
for app in ios_valor:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Agora vamos analisar um pouco a Google Play.

## Apps mais populares (por gênero) na Google Play

In [27]:
mostrar_tabela(android_valor, 5) # coluna dos downloads

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


Para fazer os cálculos, precisaremos transformar esses números em tipo `float`:

In [28]:
# Perceba que teremos que tirar os caracteres especiais para transformar os valores em float #

categorias_android = tab_freq(android_valor, 1)

for categoria in categorias_android:
    total = 0
    categoria_tam = 0
    for app in android_valor:
        categoria_app = app[1]
        if categoria_app == categoria:
            n_downloads = app[5]
            n_downloads = n_downloads.replace(',', '')
            n_downloads = n_downloads.replace('+', '')
            total += float(n_downloads)
            categoria_tam += 1
            
    media_n_downloads = total / categoria_tam
    print(categoria, ':', media_n_downloads)
    

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

Em média, aplicativos de comunicação tem o maior número de downloads. Mas, essa categoria também é influenciada por grandes apps como Whatsapp, Discord, Messenger, Telegram, etc.

In [29]:
for app in android_valor:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+' or app[5] == '500,000,000+' or app[5] == '100,000,000+'):
        
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Se tirarmos todos os apps com mais de 100 milhões de downloads, a média seria extremamente reduzida:

In [30]:
abaixo_de_100m = []

for app in android_valor:
    n_downloads = app[5]
    n_downloads = n_downloads.replace(',', '')
    n_downloads = n_downloads.replace('+', '')
    
    if (app[1] == 'COMMUNICATION') and (float(n_downloads) < 100000000):
        abaixo_de_100m.append(float(n_downloads))
    
sum(abaixo_de_100m)/ len(abaixo_de_100m)

3603485.3884615386

Esse padrão se repete em várias categorias (de vídeo, fotografia, produtividade, etc). Vamos dar uma olhada em mais algumas categorias: 

In [32]:
for app in android_valor:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

In [33]:
for app in android_valor:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+' or app[5] == '500,000,000+' or app[5] == '100,000,000+'):
        
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


Vamos tentar ter uma ideia de aplicativos entre 1.000.000 e 100.000.000 de downloads:

In [34]:
for app in android_valor:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+' or app[5] == '5,000,000+' or app[5] == '10,000,000+' or app[5] == '50,000,000+'):
        
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Quran for Android : 10,000,000+
Dictionary.com: Find Definitions for English Words : 10,000,000+
English Dictionary - Offline : 10,000,000+
Bible KJV : 5,000,000+
NOOK: Read eBooks & Magazines : 10,000,000+
Dictionary : 10,000,000+
Spanish English Translator : 10,000,000+
Dictionary - Merriam-Webster : 10,000,000+
JW Library : 10,000,000+
Oxford Dictionary of English : Free : 10,000,000+
English Hindi Dictionary : 1

Existem alguns aplicativos criados com base no Alcorão, o que sugere que criar um aplicativo em torno de um livro popular pode ser lucrativo. Pegar um livro popular e transformá-lo em um aplicativo pode ser lucrativo para a Google Play e a App Store.

## Considerações Finais

Analisamos neste projeto, dados sobre os aplicativos da App Store e Google Play a fim de recomendar um perfil de aplicativo que possa ser lucrativo para ambos os mercados.

Vimos que pegar um livro popular e transformá-lo em um aplicativo pode ser lucrativo para a Google Play e para a App Store. Como as duas plataformas já estão cheias de bibliotecas, é prciso que haja algum diferencial nesse aplicativo que será criado. Adicionar alguns recursos especiais além da versão bruta do livro pode incluir citações diárias do livro, uma versão em áudio do livro, um esquema de perguntas e respostas sobre o livro, um fórum onde as pessoas podem discutir o livro, etc.